# Benchmark testing of coherence pipeline on Movies dataset
## How to find how well coherence measure matches your manual annotators

__Introduction__: For the validation of any model adapted from a paper, it is of utmost importance that the results of benchmark testing on the datasets listed in the paper match between the actual implementation (palmetto) and gensim. This coherence pipeline has been implemented from the work done by Roeder et al. The paper can be found [here](http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf).

__Approach__ :
1. In this notebook, we'll use the Movies dataset mentioned in the paper. This dataset along with the topics on which the coherence is calculated and the gold (human) ratings on these topics can be found [here](http://139.18.2.164/mroeder/palmetto/datasets/).
2. We will then calculate the coherence on these topics using the pipeline implemented in gensim.
3. Once we have all our coherence values on these topics we will calculate the correlation with the human ratings using pearson's r.
4. We will compare this final correlation value with the values listed in the paper and see if the pipeline is working as expected.

In [1]:
from __future__ import print_function

import re
import os

from scipy.stats import pearsonr
from datetime import datetime

from gensim.models import CoherenceModel
from gensim.corpora.dictionary import Dictionary

Download the dataset (`movie.zip`) and gold standard data (`topicsMovie.txt` and `goldMovie.txt`) from the link and plug in the locations below.

In [2]:
base_dir = os.path.join(os.path.expanduser('~'), "workshop/nlp/data/")
data_dir = os.path.join(base_dir, 'wiki-movie-subset')
if not os.path.exists(data_dir):
    raise ValueError("SKIP: Please download the movie corpus.")

ref_dir = os.path.join(base_dir, 'reference')
topics_path = os.path.join(ref_dir, 'topicsMovie.txt')
human_scores_path = os.path.join(ref_dir, 'goldMovie.txt')

In [3]:
%%time

texts = []
file_num = 0
preprocessed = 0
listing = os.listdir(data_dir)

for fname in listing:
    file_num += 1
    if 'disambiguation' in fname:
        continue  # discard disambiguation and redirect pages
    elif fname.startswith('File_'):
        continue  # discard images, gifs, etc.
    elif fname.startswith('Category_'):
        continue  # discard category articles
        
    # Not sure how to identify portal and redirect pages,
    # as well as pages about a single year.
    # As a result, this preprocessing differs from the paper.
    
    with open(os.path.join(data_dir, fname)) as f:
        for line in f:
            # lower case all words
            lowered = line.lower()
            #remove punctuation and split into seperate words
            words = re.findall(r'\w+', lowered, flags = re.UNICODE | re.LOCALE)
            texts.append(words)
            
    preprocessed += 1
    if file_num % 10000 == 0:
        print('PROGRESS: %d/%d, preprocessed %d, discarded %d' % (
            file_num, len(listing), preprocessed, (file_num - preprocessed)))

PROGRESS: 10000/125384, preprocessed 9916, discarded 84
PROGRESS: 20000/125384, preprocessed 19734, discarded 266
PROGRESS: 30000/125384, preprocessed 29648, discarded 352
PROGRESS: 50000/125384, preprocessed 37074, discarded 12926
PROGRESS: 60000/125384, preprocessed 47003, discarded 12997
PROGRESS: 70000/125384, preprocessed 56961, discarded 13039
PROGRESS: 80000/125384, preprocessed 66891, discarded 13109
PROGRESS: 90000/125384, preprocessed 76784, discarded 13216
PROGRESS: 100000/125384, preprocessed 86692, discarded 13308
PROGRESS: 110000/125384, preprocessed 96593, discarded 13407
PROGRESS: 120000/125384, preprocessed 106522, discarded 13478
CPU times: user 19.8 s, sys: 9.55 s, total: 29.4 s
Wall time: 44.9 s


In [4]:
%%time

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

CPU times: user 1min 26s, sys: 1.1 s, total: 1min 27s
Wall time: 1min 27s


### Cross validate the numbers

According to the paper the number of documents should be 108,952 with a vocabulary of 1,625,124. The difference is because of a difference in preprocessing. However the results obtained are still very similar.

In [5]:
print(len(corpus))
print(dictionary)

111637
Dictionary(756837 unique tokens: [u'verplank', u'mdbg', u'shatzky', u'duelcity', u'dulcitone']...)


In [6]:
topics = []  # list of 100 topics
with open(topics_path) as f:
    topics = [line.split() for line in f if line]
len(topics)

100

In [7]:
human_scores = []
with open(human_scores_path) as f:
    for line in f:
        human_scores.append(float(line.strip()))
len(human_scores)

100

### Deal with any vocabulary mismatch.

In [8]:
# We first need to filter out any topics that contain terms not in our dictionary
# These may occur as a result of preprocessing steps differing from those used to
# produce the reference topics. In this case, this only occurs in one topic.
invalid_topic_indices = set(
    i for i, topic in enumerate(topics)
    if any(t not in dictionary.token2id for t in topic)
)
print("Topics with out-of-vocab terms: %s" % ', '.join(map(str, invalid_topic_indices)))
usable_topics = [topic for i, topic in enumerate(topics) if i not in invalid_topic_indices]

Topics with out-of-vocab terms: 72


### Start off with u_mass coherence measure.

In [ ]:
%%time

cm = CoherenceModel(topics=usable_topics, corpus=corpus, dictionary=dictionary, coherence='u_mass')
u_mass = cm.get_coherence_per_topic()
print("Calculated u_mass coherence for %d topics" % len(u_mass))

Calculated u_mass coherence for 99 topics
CPU times: user 7.22 s, sys: 141 ms, total: 7.36 s
Wall time: 7.38 s


### Start c_v coherence measure
This is expected to take much more time since `c_v` uses a sliding window to perform probability estimation and uses the cosine similarity indirect confirmation measure.

In [ ]:
%%time

cm = CoherenceModel(topics=usable_topics, texts=texts, dictionary=dictionary, coherence='c_v')
c_v = cm.get_coherence_per_topic()
print("Calculated c_v coherence for %d topics" % len(c_v))

### Start c_uci and c_npmi coherence measures
c_v and c_uci and c_npmi all use the boolean sliding window approach of estimating probabilities. Since the `CoherenceModel` caches the accumulated statistics, calculation of c_uci and c_npmi are practically free after calculating c_v coherence. These two methods are simpler and were shown to correlate less with human judgements than c_v but more so than u_mass.

In [ ]:
%%time

cm.coherence = 'c_uci'
c_uci = cm.get_coherence_per_topic()
print("Calculated c_uci coherence for %d topics" % len(c_uci))

In [ ]:
%%time

cm.coherence = 'c_npmi'
c_npmi = cm.get_coherence_per_topic()
print("Calculated c_npmi coherence for %d topics" % len(c_npmi))

In [ ]:
final_scores = [
    score for i, score in enumerate(human_scores)
    if i not in invalid_topic_indices
]
len(final_scores)

The [values in the paper](http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf) were:

__`u_mass` correlation__ : 0.093

__`c_v` correlation__    : 0.548

__`c_uci` correlation__  : 0.473

__`c_npmi` correlation__ : 0.438

Our values are also very similar to these values which is good. This validates the correctness of our pipeline.

In [ ]:
for our_scores in (u_mass, c_v, c_uci, c_npmi):
    print(pearsonr(our_scores, final_scores)[0])

### Where do we go now?

- Preprocessing can be improved for this notebook by following the exact process mentioned in the reference paper. Specifically: _All corpora as well as the complete Wikipedia used as reference corpus are preprocessed using lemmatization and stop word removal. Additionally, we removed portal and category articles, redirection and disambiguation pages as well as articles about single years._ Right now it differs only in not removing redirect and portal pages and pages about single years.
- The time required for completing all of these operations can be improved a lot by cythonising them.